In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

In [2]:
from math import pow
import random
import json

from roaringbitmap import RoaringBitmap, MultiRoaringBitmap

class RoundRobinSamplingBitmap:
    """A bitmap sketch with fixed max size via variable sampling"""    
    
    MAX_uint32_t = 0xFFFFFFFF ## TODO find a more pythonic way to assert this
    
    # TODO to be stict to remove chance of unitialized member vars
    # TODO having a version that requires everything we need for the results of intersection, union, etc
    # TODO consider dealing with negatives in pruning
    def __init__(self, maxSamples):        
        self.maxSamples = maxSamples
        self.curSampleRate = 1   # the rate is 1/curSampleRate, so if the value is 4, it is 1/4th. start at 1/1
        self.rbm = RoaringBitmap()  

    def copy(self):
        result = RoundRobinSamplingBitmap(maxSamples=self.maxSamples)
        
        result.rbm = self.rbm.copy()
        result.curSampleRate = self.curSampleRate
        return result
        
    def _checkSize(self):          
        # first, prune out anything beyond the max because they're not in the sample
        self._prune()

        # now, as long have more than the maxSamples number, we need to grow our sampling rate and re-prune
        while len(self.rbm) > self.maxSamples:
            self.curSampleRate *= 2 # TODO consider different sampleRate scaling instead of 2
            self._prune()

    def _getCurMaxAllowed(self):
        return int(self.MAX_uint32_t / self.curSampleRate)
    
    def _prune(self):          
        curMaxAllowed = self._getCurMaxAllowed()
        if self.rbm.max() > curMaxAllowed:
            self.rbm = self.rbm.clamp(0, curMaxAllowed)

    def add(self, i):          
        if i < 0: raise RuntimeError("cannot yet support negative id values")      
        self.rbm.add(i)
        self._checkSize()        
        return i
    
    def _performBitmapOp(self, other, opStr):     
                
        ## currently need to ensure the instances have the same maxSamples init values
        if self.maxSamples != other.maxSamples:
            raise RuntimeError("you cannot mix instances with different initial maxSamples values")
        ## TODO figure out how to extrapolate, if possible, to allow different init values
                     
        result = RoundRobinSamplingBitmap(maxSamples=self.maxSamples)
               
        result.rbm = getattr(self.rbm, opStr)(other.rbm)
        result.curSampleRate = max(self.curSampleRate, other.curSampleRate)
        
        result._checkSize()
        return result

    
    ## the following should work logically but havent really been tested with real data much
    def intersection(self, other):        
        return self._performBitmapOp(other, "intersection")

    def union(self, other):        
        return self._performBitmapOp(other, "union")
                 
    def difference(self, other):        
        return self._performBitmapOp(other, "difference")
    ## the above should work logically but havent really been tested with real data much

    
    def estimatedCardinality(self):
        return len(self.rbm)*self.curSampleRate
        
    def _getCurMaxAllowed(self):
        return int(self.MAX_uint32_t / self.curSampleRate)
    
    ## TODO deprecate this in favor of json dumps
    def dumps(self):
        state = {
            "estCard" : int(self.estimatedCardinality()),
            "curSampleRate" : self.curSampleRate,
            "curMax" : self.rbm.max(),                        
            "getCurMaxAllowed" : self._getCurMaxAllowed(),                        
        }        
        return json.dumps(state)
    
    def errPct(self, target):
        precision = 3
        mult = pow(10,precision+1)
        return int(mult*(self.estimatedCardinality()-target)/target)/mult
        

print(RoundRobinSamplingBitmap)


<class '__main__.RoundRobinSamplingBitmap'>


In [48]:
import time
random.seed(time.time()*1000)


## this cell creates a rbmrr or increasing size and saves off snapshot copies along the way.
## also, for each addition it increments a count for the observed value of error between the 
## actual and estimated cardinality.

rbmrr = RoundRobinSamplingBitmap(maxSamples=1000)

iterations = 1000*1000 #100*1000*1000

snapshots = {}
NUM_SNAPSHOTS = 50
errHistogram = {} # TODO consider using defaultdict

for i in range(1,iterations+1):
        
    rbmrr.add(random.randint(0,rbmrr.MAX_uint32_t))
    
    errPct = rbmrr.errPct(i)
    if errPct not in errHistogram:
        errHistogram[errPct] = 1
    else:
        errHistogram[errPct] += 1
            
    if (NUM_SNAPSHOTS > 0 and i % (iterations/NUM_SNAPSHOTS) == 0): 
        print(i, rbmrr.errPct(i), rbmrr.dumps())
        snapshots[i] = rbmrr.copy()



20000 0.0224 {"estCard": 20448, "curSampleRate": 32, "curMax": 134083804, "getCurMaxAllowed": 134217727}
40000 0.0016 {"estCard": 40064, "curSampleRate": 64, "curMax": 66976737, "getCurMaxAllowed": 67108863}
60000 0.025 {"estCard": 61504, "curSampleRate": 64, "curMax": 66984409, "getCurMaxAllowed": 67108863}
80000 0.0064 {"estCard": 80512, "curSampleRate": 128, "curMax": 33545328, "getCurMaxAllowed": 33554431}
100000 -0.0067 {"estCard": 99328, "curSampleRate": 128, "curMax": 33545328, "getCurMaxAllowed": 33554431}
120000 0.0058 {"estCard": 120704, "curSampleRate": 128, "curMax": 33545328, "getCurMaxAllowed": 33554431}
140000 -0.0637 {"estCard": 131072, "curSampleRate": 256, "curMax": 16758263, "getCurMaxAllowed": 16777215}
160000 -0.0608 {"estCard": 150272, "curSampleRate": 256, "curMax": 16758263, "getCurMaxAllowed": 16777215}
180000 -0.0613 {"estCard": 168960, "curSampleRate": 256, "curMax": 16758263, "getCurMaxAllowed": 16777215}
200000 -0.0528 {"estCard": 189440, "curSampleRate": 2

In [49]:
## this cell walks the errHistogram twice to compute percentiles on observed estimation error


## the first traversal is to increment a total count and to compute a new dictory keyed 
## on the abs value of observed differences between the actual and estimated cardinality.
## it also sums the counts overall so we have the total amount.

absHist, totalCount = {}, 0
for errPct, count in errHistogram.items():
    totalCount += count
    if abs(errPct) not in absHist:
        absHist[abs(errPct)] =0

    absHist[abs(errPct)] += count
    
##  the second time walks the abs of the error to show cumulative counts.
    
cumulativeCount = 0
for absErrPct, count in sorted(absHist.items()):
    cumulativeCount += count
    print(absErrPct, count, cumulativeCount/totalCount)
    
    
## with a recent run, for example, the max error was 3.88%. that is, for all keys 
## values added sequentially from 1 and 100,000,000 using at most 10,000 samples,
## the max error was 3.88% and that occured only 488 times. the p95 of error for the
## same keys was 3.49%

    

0.0 3594 0.003594
0.0001 2491 0.006085
0.0002 2273 0.008358
0.0003 2191 0.010549
0.0004 2077 0.012626
0.0005 2110 0.014736
0.0006 2188 0.016924
0.0007 2101 0.019025
0.0008 2100 0.021125
0.0009 1910 0.023035
0.001 1915 0.02495
0.0011 1950 0.0269
0.0012 2026 0.028926
0.0013 2225 0.031151
0.0014 2533 0.033684
0.0015 2605 0.036289
0.0016 2649 0.038938
0.0017 2515 0.041453
0.0018 2744 0.044197
0.0019 2959 0.047156
0.002 2995 0.050151
0.0021 3111 0.053262
0.0022 3402 0.056664
0.0023 3520 0.060184
0.0024 3593 0.063777
0.0025 3209 0.066986
0.0026 2995 0.069981
0.0027 3056 0.073037
0.0028 3111 0.076148
0.0029 3087 0.079235
0.003 2928 0.082163
0.0031 3147 0.08531
0.0032 3335 0.088645
0.0033 3090 0.091735
0.0034 2910 0.094645
0.0035 2925 0.09757
0.0036 3352 0.100922
0.0037 3659 0.104581
0.0038 3518 0.108099
0.0039 3433 0.111532
0.004 3371 0.114903
0.0041 2949 0.117852
0.0042 2777 0.120629
0.0043 2634 0.123263
0.0044 2664 0.125927
0.0045 2677 0.128604
0.0046 2807 0.131411
0.0047 2364 0.133775
0.00

0.0626 653 0.95403
0.0627 601 0.954631
0.0628 591 0.955222
0.0629 608 0.95583
0.063 677 0.956507
0.0631 651 0.957158
0.0632 668 0.957826
0.0633 656 0.958482
0.0634 663 0.959145
0.0635 666 0.959811
0.0636 629 0.96044
0.0637 592 0.961032
0.0638 571 0.961603
0.0639 576 0.962179
0.064 551 0.96273
0.0641 561 0.963291
0.0642 588 0.963879
0.0643 587 0.964466
0.0644 558 0.965024
0.0645 592 0.965616
0.0646 592 0.966208
0.0647 604 0.966812
0.0648 561 0.967373
0.0649 552 0.967925
0.065 594 0.968519
0.0651 606 0.969125
0.0652 602 0.969727
0.0653 615 0.970342
0.0654 629 0.970971
0.0655 637 0.971608
0.0656 632 0.97224
0.0657 613 0.972853
0.0658 645 0.973498
0.0659 600 0.974098
0.066 593 0.974691
0.0661 551 0.975242
0.0662 537 0.975779
0.0663 534 0.976313
0.0664 533 0.976846
0.0665 531 0.977377
0.0666 537 0.977914
0.0667 519 0.978433
0.0668 469 0.978902
0.0669 388 0.97929
0.067 345 0.979635
0.0671 309 0.979944
0.0672 301 0.980245
0.0673 274 0.980519
0.0674 274 0.980793
0.0675 291 0.981084
0.0676 287 

In [50]:

## here we use the snapshots to compare error in intersect estimates

largestSnapshotSize = max(snapshots.keys())
largestSnapshot = snapshots[largestSnapshotSize]

print("intersecting with the largest snapshot with %d entries\n" % largestSnapshotSize)

intersectErrPcts = []
for k,v in snapshots.items():
    intersect = largestSnapshot.intersection(v)
    intersectErrPcts.append(intersect.errPct(k))
    print(k, intersect.errPct(k), intersect.dumps())





intersecting with the largest snapshot with 1000000 entries

20000 -0.0784 {"estCard": 18432, "curSampleRate": 1024, "curMax": 4095053, "getCurMaxAllowed": 4194303}
40000 -0.104 {"estCard": 35840, "curSampleRate": 1024, "curMax": 4095053, "getCurMaxAllowed": 4194303}
60000 -0.1296 {"estCard": 52224, "curSampleRate": 1024, "curMax": 4095053, "getCurMaxAllowed": 4194303}
80000 -0.04 {"estCard": 76800, "curSampleRate": 1024, "curMax": 4095053, "getCurMaxAllowed": 4194303}
100000 -0.0374 {"estCard": 96256, "curSampleRate": 1024, "curMax": 4125106, "getCurMaxAllowed": 4194303}
120000 0.024 {"estCard": 122880, "curSampleRate": 1024, "curMax": 4125106, "getCurMaxAllowed": 4194303}
140000 0.024 {"estCard": 143360, "curSampleRate": 1024, "curMax": 4189395, "getCurMaxAllowed": 4194303}
160000 0.0368 {"estCard": 165888, "curSampleRate": 1024, "curMax": 4189395, "getCurMaxAllowed": 4194303}
180000 0.0126 {"estCard": 182272, "curSampleRate": 1024, "curMax": 4189395, "getCurMaxAllowed": 4194303}
200

In [54]:
mrb = MultiRoaringBitmap([v.rbm.freeze() for v in snapshots.values()])
print("avg snapshot size: %d bytes" % (mrb.bufsize()/len(snapshots.values())))

snapshotBufsize = [(k,v.curSampleRate,MultiRoaringBitmap([v.rbm]).bufsize()) for (k,v) in snapshots.items()]
sorted(snapshotBufsize)

avg snapshot size: 7818 bytes


[(20000, 32, 27616),
 (40000, 64, 23808),
 (60000, 64, 31456),
 (80000, 128, 18400),
 (100000, 128, 20544),
 (120000, 128, 22368),
 (140000, 256, 11360),
 (160000, 256, 11648),
 (180000, 256, 11808),
 (200000, 256, 12064),
 (220000, 256, 12256),
 (240000, 256, 12512),
 (260000, 256, 12544),
 (280000, 512, 6368),
 (300000, 512, 6368),
 (320000, 512, 6368),
 (340000, 512, 6368),
 (360000, 512, 6368),
 (380000, 512, 6400),
 (400000, 512, 6464),
 (420000, 512, 6464),
 (440000, 512, 6464),
 (460000, 512, 6464),
 (480000, 512, 6464),
 (500000, 1024, 3296),
 (520000, 1024, 3296),
 (540000, 1024, 3296),
 (560000, 1024, 3296),
 (580000, 1024, 3296),
 (600000, 1024, 3296),
 (620000, 1024, 3328),
 (640000, 1024, 3360),
 (660000, 1024, 3360),
 (680000, 1024, 3360),
 (700000, 1024, 3392),
 (720000, 1024, 3392),
 (740000, 1024, 3456),
 (760000, 1024, 3488),
 (780000, 1024, 3520),
 (800000, 1024, 3552),
 (820000, 1024, 3584),
 (840000, 1024, 3680),
 (860000, 1024, 3712),
 (880000, 1024, 3744),
 (9000